# Amazon Search Project
---
**Daniel Crews**
and
**Brandon Ginn**

With this project we attempting to provide a tool that will allow people to query reviews of Amazon food products.  By providing a search of the reviews, rather than item titles, people can use descriptors like 'spicy' to get results from others who pointed out these attributes in the product.

We wanted to investigate the impact of stopwords being included in the inverted index and the performance of TF-IDF compared to BM25.


## Amazon Food Reviews Corpus

In [1]:
from collections import Counter

import csv
import re

f = open('data/amazon/reviews.csv')
csv_f = csv.reader(f)

review_id = []
review_profile = []
review_text = []

for row in csv_f:
    review_id.append(row[1])
    review_profile.append(row[3])
    review_text.append(row[9])

### text_print function displays review text as HTML

In [2]:
from IPython.core.display import display, HTML

def text_print(text):
    display(HTML(text))

In [3]:
def print_results(results, n, text, rev_id, head=True):
    from IPython.display import display
    
    if head:    
        print('\nTop %d from recall set of %d items:' % (n,len(results)))
        for r in results[:n]:
            print('Item: %s'%(rev_id[r[1]]))
            print('Score: %0.2f'%(r[0]))
            text_print(text[r[1]])
    else:
        print('\nBottom %d from recall set of %d items:' % (n,len(results)))
        for r in results[-n:]:
            print('\t%0.2f - %s'%(r[0],text[r[1]]))

## TF-IDF
---
The corpus of review text contained HTML, so we removed this when creating the inverted index.  We didn't want to strip the text of the HTML when reading from the CSV so the links put in by the reviewer are maintained.


In [4]:
import math

def idf(term, idx, n):
    return math.log( float(n) / ((1 + (len(idx[term])))))

In [5]:
def create_inverted_index(corpus):
    idx={}
    for i, doc in enumerate(corpus):
        for word in doc.split():
            word = re.sub("<.*?>","",word) # Strip out HTML
            word = word.lower()
            word = word.replace('[','')
            word = word.replace(']','')
            word = word.replace('.','')
            if word == "/><br": 
                continue
            if word in idx:
                if i in idx[word]:
                    idx[word][i] += 1
                else:
                    idx[word][i] = 1
            else:
                # Add term
                idx[word] = {i:1}
    return idx

In [6]:
def get_results_tfidf(qry, idx, n):
    score = Counter()
    for term in qry.split():
        if term in idx:
            i = idf(term, idx, n)
            for doc in idx[term]:
                score[doc] += idx[term][doc] * i
        
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            results.append([x[1],x[0]])
    
    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results

## Results before the removal of stopwords.

We wanted to see what kind of impact the stopwords made on our search results.  Words like 'the' and 'I' are given very low scores, but given how long some of the reviews are, the impact could really add up.

In [16]:
idx = create_inverted_index(review_text)

### The 40 most common words before stopword filtering.

In [8]:
import pandas as pd
from bokeh.plotting import output_notebook, show
from bokeh.charts import Bar
from bokeh.charts.attributes import CatAttr
#from bokeh.models import ColumnDataSource

df = pd.DataFrame({'term':[x for x in idx.keys()],'freq':[len(x) for x in idx.values()]})

output_notebook(hide_banner=True)
p = Bar(df.sort_values('freq', ascending=False)[:40], label=CatAttr(columns=['term'], sort=False), values='freq',
        plot_width=800, plot_height=400)
show(p)

### How the term frequency impacts the scoring of certain words.

In [9]:
print("Dog: " + str(idf("dog", idx, len(review_text))))
print("Hot: " + str(idf("hot", idx, len(review_text))))
print("Ramen: " + str(idf("ramen", idx, len(review_text))))
print("Spicy: " + str(idf("spicy", idx, len(review_text))))
print("The: " + str(idf("the", idx, len(review_text))))
print("And: " + str(idf("and", idx, len(review_text))))
print("I: " + str(idf("i", idx, len(review_text))))

Dog: 2.83018339799
Hot: 3.12484664969
Ramen: 6.44163812732
Spicy: 4.28726114137
The: 0.187904837376
And: 0.232291084636
I: 0.261493510525


### Getting the updated top five results
---
While many of the returned results are the same, the scores more accurately reflect the value of the review.  On average, a long review loses around 15 points with the removal of the stopwords.


In [10]:
result = get_results_tfidf("the spicy noodles and soup", idx, len(review_text))
print_results(result, 5, review_text, review_id)


Top 5 from recall set of 535923 items:
Item: B000LQORDE
Score: 149.84


Item: B000LQLV7E
Score: 115.53


Item: B002GDH5Y8
Score: 107.78


Item: B000LQORDE
Score: 106.28


Item: B000V6FTZO
Score: 94.68


## Now filtering out the stopwords
---
All 40 of the most frequent words were part of NLTK stopwords before filtering.  After removing them, we have some terms more relevant to food start showing up, such as flavor. 

In [24]:
from nltk.corpus import stopwords

for word in stopwords.words('english'):
    if word in idx: del idx[word]

### The 40 most common words after stopword filtering.

In [25]:
import pandas as pd
from bokeh.plotting import output_notebook, show
from bokeh.charts import Bar
from bokeh.charts.attributes import CatAttr
#from bokeh.models import ColumnDataSource

df = pd.DataFrame({'term':[x for x in idx.keys()],'freq':[len(x) for x in idx.values()]})

output_notebook(hide_banner=True)
p = Bar(df.sort_values('freq', ascending=False)[:40], label=CatAttr(columns=['term'], sort=False), values='freq',
        plot_width=800, plot_height=400)
show(p)

### Getting the updated top five results
---


In [26]:
result = get_results_tfidf("the spicy noodles and soup", idx, len(review_text))
print_results(result, 5, review_text, review_id)


Top 5 from recall set of 16315 items:
Item: B000LQORDE
Score: 130.79


Item: B000LQLV7E
Score: 105.85


Item: B002GDH5Y8
Score: 99.54


Item: B000LQORDE
Score: 96.78


Item: B000H23ZE4
Score: 87.87


## Implementing BM25 instead of TF-IDF
---
We wanted to see how effective BM25 would be compared to TF-IDF.  By testing a few queries on both, we have found it returns better overall results.  Particularly in the length of the queries.  TF-IDF tended to favor much longer reviews, but these often less relevant than the short reviews obtained by BM25.


In [27]:
def get_results5(qry, corpus, k1=1.5, b=0.75):
    n = len(corpus)
    d = [len(x.split()) for x in corpus]
    d_avg = float(sum(d)) / len(d)                
    score = Counter()
    for term in qry.split():
        if term in idx:
            i = idf(term, idx, n)
            for doc in idx[term]:
                f = float(idx[term][doc])
                score[doc] += i * (( f * (k1 + 1) ) / (f + k1 * (1 - b + (b * (float(d[doc]) / d_avg)))))
        
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            # output [0] score, [1] doc_id
            results.append([x[1],x[0]])
    results.sort(key=lambda x: x[0], reverse=True)
    return results;

def idf(term, idx, n):
    return math.log( float(n) / (1 + len(idx[term])))

In [28]:
results = get_results5('the spicy noodles and soup', review_text, k1=1.5, b=0.75)
print_results(results, 5, review_text, review_id)


Top 5 from recall set of 16315 items:
Item: B000E8VXZ4
Score: 24.82


Item: B000LQORES
Score: 21.47


Item: B004LLD3MQ
Score: 20.86


Item: B000FFRTYK
Score: 20.84


Item: B000LQORDE
Score: 20.51
